This open-source software aids in identifying the ideal ratio of each spectral data component for the reconstruction of a sample when only its spectral information is available.This is a sample to reconstruct [Western University](https://www.uwo.ca/index.html)'s color which Purple. It could be applied to any other color.

In [3]:
#install required libraries

!pip install GPyOpt
!pip install pyDOE
!pip install scikit-optimize

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import GPyOpt
from pyDOE import lhs
from skopt import gp_minimize
from skopt.space import Real

If the step size in the reflectance data is not either 10 or 1 nanometer, we should employ interpolation method to achieve that. If you don't have that problem, skip this step.

In [ ]:
data = np.array(pd.read_excel('Purple.xlsx',usecols='A:E')) #slime green

x, y = data[:,0], data[:,4] #extracting raw x and Y values from 1st and second column of sheet

LOWER_BOUND = 400  #smallest wavelength value to interpolate
UPPER_BOUND = 700  #largest wavelength value to interpolate
numOfValues = int((UPPER_BOUND - LOWER_BOUND)/10) + 1 #number of values to produce within the bounds


x_space = np.linspace(LOWER_BOUND,UPPER_BOUND, numOfValues ) #whole number array list w/ +1 increments
y_space = np.interp(x_space, x, y) #interpolating using whole number values and x, y data from spreedsheet


plt.plot(x_space, y_space, label='interpolated reflectance')
plt.plot(x,y, label='prior to interpolation')
plt.title('Interpolated red petg')
plt.xlabel('Wave Length')
plt.ylabel('% Reflectance')
plt.show()

In [ ]:
#import your data here:

#these are always the same
xyzbar = np.array(pd.read_excel('XYZ.xlsx',usecols='C:E'))

E = np.array(pd.read_excel('XYZ.xlsx',usecols='B'))

#Reflectances
data = np.array(pd.read_excel('clean-data-10nm.xlsx', usecols='A:J'))

In [ ]:
wavelength = data[:,0]

#you could add any colors that you have
white = data[:,1]
red_petg = data[:,2]
black = data[:,4]
dark_purple = data[:,5] #light purple
gray = data[:,6]
blue_pla = data[:,7]
cream = data[:,8]
Slime_green = data[:,9]


#purple PET-G
target = data[:,3]

Plot the data

In [ ]:
x = wavelength

plt.plot(x, target, 'purple', label='target', linestyle= '-.', markersize='7')  # purple
plt.plot(x, white, 'black', label='white', markersize='7')  # white
plt.plot(x, red_petg, 'red', label='red_petg', markersize='7')  # red_petg
plt.plot(x, black, 'black', label='black', markersize='7')  # black
plt.plot(x, dark_purple, 'magenta', label='dark_purple', markersize='7')  # dark_purple
plt.plot(x, gray, 'gray', label='gray', markersize='7')  # gray
plt.plot(x, blue_pla, 'blue', label='blue_pla', markersize='7')  # blue_pla
plt.plot(x, cream, 'orange', label='cream', markersize='7')  # cream
plt.plot(x, Slime_green, 'green', label='Slime_green', markersize='7')  # Slime_green

plt.xlabel('Wavelength')
plt.ylabel('% Reflectance')
plt.title('Reflectance of Samples Before Interpolation')
plt.legend(loc='best')
plt.grid(False)
plt.show()

A function to convert reflectance data to CIELAB

In [ ]:
def xyz(E, xbar, ybar, zbar, R):

  E = np.array(E)
  xbar = np.array(xbar)
  ybar = np.array(ybar)
  zbar = np.array(zbar)
  R = np.array(R)

  K = 100 / sum(E * ybar)
  Xn = K * sum(E * xbar)
  Yn = K * sum(E * ybar)
  Zn = K * sum(E * zbar)
  X = K * sum(E * xbar * R)
  Y = K * sum(E * ybar * R)
  Z = K * sum(E * zbar * R)
  L = 116 * (abs(Y / Yn) ** (1 / 3)) - 16
  a = 500 * (abs(X / Xn) * (1 / 3) - abs(Y / Yn) * (1 / 3))
  b = np.where(np.abs(Y / Yn) < 1e-10, 0, 200 * (np.abs(Y / Yn) ** (1 / 3) - np.abs(Z / Zn) ** (1 / 3)))
  XYZ = [X, Y, Z]
  lab = [L, a, b]
  return lab

The Objective Function

In [ ]:
def objective(proportions):

    proportions = (proportions.reshape(5, 1)) #this could change based on the number of colors


    #the target color might be made of more or less number of colors. This should change depending on your case
    predicted_color = (proportions[0] * white +
                       proportions[1] * red_petg +
                       proportions[2] * dark_purple +
                       proportions[3] * black +
                       proportions[4] * blue_pla)

    #shape = predicted_color.shape
    #print(shape)

    # RMS Error
    lse_error = np.sqrt(np.mean((predicted_color - target) ** 2))

    # ΔE Error
    target_lab = xyz(E, xyzbar[:,0],xyzbar[:,1],xyzbar[:,2],target)
    L_target, a_target, b_target = target_lab

    predicted_lab = xyz(E, xyzbar[:,0],xyzbar[:,1],xyzbar[:,2],predicted_color)
    L_predict, a_predict, b_predict = predicted_lab

    delta_e = (sum((np.array(predicted_lab).real - np.array(target_lab).real) ** 2)) ** (0.5)
    delta_e = np.nan_to_num(delta_e, nan=0)
    delta_e_mean=(np.mean(delta_e))


    # Combine the two error metrics as needed
    combined_objective = 0.9 * lse_error + 0.1 * delta_e_mean


    return combined_objective


Predicted spectrum

In [ ]:
def predictedSpectrum (estimation):

  predicted_spectrum = (estimation[0] * white +
                      estimation[1] * red_petg +
                      estimation[2] * dark_purple +
                      estimation[3] * black +
                      estimation[4] * blue_pla)
  return predicted_spectrum


A function to define all sorts of evaluation metrics you might need. Here, RMS and Delta E are utilized:

In [ ]:
def errors(pred_spec):
  rms = np.sqrt(np.mean((pred_spec - target)**2))

  target_lab = xyz(E, xyzbar[:,0],xyzbar[:,1],xyzbar[:,2],target)
  L_target, a_target, b_target = target_lab

  predicted_lab = xyz(E, xyzbar[:,0],xyzbar[:,1],xyzbar[:,2],pred_spec)
  L_predict, a_predict, b_predict = predicted_lab

  delta_e =  (sum((np.array(predicted_lab).real - np.array(target_lab).real) ** 2)) ** (0.5)
  delta_e = np.nan_to_num(delta_e, nan=0)

  result = print(f'RMS = {np.round(rms,4)}\ndelta E = {np.round(np.mean(delta_e),4)}')

  return result

# **1.GPyOpt optimization method**

Bounds

In [ ]:
bounds = [{'name': 'proportion_1', 'type': 'continuous', 'domain': (0, 1)},
          {'name': 'proportion_2', 'type': 'continuous', 'domain': (0, 1)},
          {'name': 'proportion_3', 'type': 'continuous', 'domain': (0, 1)},
          {'name': 'proportion_4', 'type': 'continuous', 'domain': (0, 1)},
          {'name': 'proportion_5', 'type': 'continuous', 'domain': (0, 1)}]


domain_values = [bound['domain'] for bound in bounds]
print(np.shape(domain_values))
print(domain_values)

1.1.The Optimization (EI, CMA)

In [ ]:
initial_design_numdata = 5 # Number of initial data points
initial_data = lhs(len(bounds), samples=initial_design_numdata)
#print(f'Initial data =\n {initial_data}')


problem = GPyOpt.methods.BayesianOptimization(
    f= objective,
    domain= bounds,
    X=initial_data,  # Number of initial data points
    acquisition_type='EI',    # Expected Improvement
    acquisition_optimizer='CMA',
    maximize= False            # Set to True if maximizing the objective function
)
max_iter = 250
problem.run_optimization(max_iter=max_iter)

# Estimated proportions
estimated= problem.x_opt
estimated_value = problem.fx_opt

The error

In [ ]:
# Print the estimated proportions
print(f'Estimated Proportions:\n{estimated}')
predicted_spectrum_0 = predictedSpectrum(estimated)
evaluation = errors(predicted_spectrum_0)

1.2.The Optimization (EI, LBFGS)

In [ ]:
initial_design_numdata = 5 # Number of initial data points
initial_data = lhs(len(bounds), samples=initial_design_numdata)


problem1 = GPyOpt.methods.BayesianOptimization(
    f= objective,
    domain= bounds,
    X=initial_data,  # Number of initial data points
    acquisition_type='EI',    # Expected Improvement
    acquisition_optimizer='lbfgs',
    maximize= False            # Set to True if maximizing the objective function
)
max_iter = 250
problem1.run_optimization(max_iter=max_iter)

# Estimated proportions
estimated1= problem1.x_opt
estimated_value1 = problem1.fx_opt

Errors

In [ ]:
# Print the estimated proportions
print(f'Estimated Proportions1:\n{estimated1}')
predicted_spectrum_1 = predictedSpectrum(estimated1)
evaluation = errors(predicted_spectrum_1)

1.3. The optimization (MPI,CMA)

In [ ]:
initial_design_numdata = 5 # Number of initial data points
initial_data = lhs(len(bounds), samples=initial_design_numdata)


problem2 = GPyOpt.methods.BayesianOptimization(
    f= objective,
    domain= bounds,
    X=initial_data,  # Number of initial data points
    acquisition_type='MPI',    # Expected Improvement
    acquisition_optimizer='CMA',
    maximize= False            # Set to True if maximizing the objective function
)
max_iter = 250
problem2.run_optimization(max_iter=max_iter)

# Estimated proportions
estimated2= problem2.x_opt
estimated_value2 = problem2.fx_opt

Error

In [ ]:
# Print the estimated proportions
print(f'Estimated Proportions:\n{estimated2}')
predicted_spectrum_2 = predictedSpectrum(estimated2)
evaluation = errors(predicted_spectrum_2)

1.4.The optimization (MPI, LBFGS)

In [ ]:
initial_design_numdata = 5 # Number of initial data points
initial_data = lhs(len(bounds), samples=initial_design_numdata)


problem3 = GPyOpt.methods.BayesianOptimization(
    f= objective,
    domain= bounds,
    X=initial_data,  # Number of initial data points
    acquisition_type='MPI',    # Expected Improvement
    acquisition_optimizer='lbfgs',
    maximize= False            # Set to True if maximizing the objective function
)
max_iter = 250
problem3.run_optimization(max_iter=max_iter)

# Estimated proportions
estimated3= problem3.x_opt
estimated_value3 = problem3.fx_opt

The Error

In [ ]:
# Print the estimated proportions
print(f'Estimated Proportions:\n{estimated3}')
predicted_spectrum_3 = predictedSpectrum(estimated3)
evaluation3 = errors(predicted_spectrum_3)

# 2.Optimization using Bayesian Optimization in Scikit-Learn library

In [ ]:
def objective(proportions):


  predicted_color = (proportions[0] * white +
                       proportions[1] * red_petg +
                       proportions[2] * dark_purple +
                       proportions[3] * black +
                       proportions[4] * blue_pla)

  # RMS Error
  lse_error = np.sqrt(np.mean((predicted_color - target) ** 2))

  # ΔE Error
  target_lab = xyz(E, xyzbar[:,0],xyzbar[:,1],xyzbar[:,2],target)
  L_target, a_target, b_target = target_lab

  predicted_lab = xyz(E, xyzbar[:,0],xyzbar[:,1],xyzbar[:,2],predicted_color)
  L_predict, a_predict, b_predict = predicted_lab

  delta_e = (sum((np.array(predicted_lab).real - np.array(target_lab).real) ** 2)) ** (0.5)
  delta_e = np.nan_to_num(delta_e, nan=0)
  delta_e_mean=(np.mean(delta_e))


  # Combine the two error metrics as needed
  combined_objective = 0.9 * lse_error + 0.1 * delta_e_mean


  return combined_objective

The space and initial guess

In [ ]:
space = [Real(0,1, name='prop1'), Real(0,1, name='prop2'), Real(0,1, name='prop3'),
         Real(0,1, name='prop4'), Real(0,1, name='prop5')]

#the initial proportion of each color in the blend to start the optimization with
guess = [0.12, 0.25 ,0.09, 0.04, 0.5]

Optimization

In [ ]:
result4 = gp_minimize(
    objective,
    space,
    x0= guess,
    n_calls = 250,
    random_state = 42
)

estimated4 = result4.x
print(estimated4)

Error

In [ ]:
print(f'Estimated Proportions: {estimated4}')
predicted_spectrum_4 = predictedSpectrum(estimated4)
evaluation4 = errors(predicted_spectrum_4)

# 3.Optimization methods using SciPy library

3.1. L-BFGS-B method

In [4]:
bounds = [(0,1)]*5
result5 = minimize(objective,guess,bounds= bounds, method='L-BFGS-B')
estimated5 = result5.x

Error

In [ ]:
print(f'Estimated Proportions:\n{estimated5}')
predicted_spectrum_5 = predictedSpectrum(estimated5)
evaluation5 = errors(predicted_spectrum_5)

3.2. SLSQP method

In [ ]:
result6 = minimize(objective,guess,bounds= bounds, method='SLSQP')
estimated6 = result6.x

Error

In [ ]:
print(f'Estimated Proportions: {estimated6}')
predicted_spectrum_6 = predictedSpectrum(estimated6)
evaluation6 = errors(predicted_spectrum_6)

3.3. Nelder-Mead method

In [ ]:
result7 = minimize(objective,guess,bounds= bounds, method='Nelder-Mead')
estimated7 = result7.x
print(estimated7)

Error

In [ ]:
print(f'Estimated Proportions:\n{estimated7}')
predicted_spectrum_7 = predictedSpectrum(estimated7)
evaluation7 = errors(predicted_spectrum_7)

Now, we plot the results to compare it to the target spectra

In [ ]:
x = wavelength
plt.plot(x, target, 'purple', label='Target color',linestyle='--', markersize='7')  # purple
plt.plot(x, predicted_spectrum_0, 'r', label='EI,CMA', markersize='7')
plt.plot(x, predicted_spectrum_1, 'b', label='EI,LBFGS', markersize='7')
plt.plot(x, predicted_spectrum_2, 'g', label='MPI,CMA', markersize='7')
plt.plot(x, predicted_spectrum_3, 'gray', label='MPI,LBFGS', markersize='7')
plt.plot(x, predicted_spectrum_4, 'orange', label='Bayesian Optimization', markersize='7')
plt.plot(x, predicted_spectrum_5, 'yellow', label='L-BFGS-B', markersize='7')
plt.plot(x, predicted_spectrum_6, 'cyan', label='SLSQP', markersize='7')
plt.plot(x, predicted_spectrum_7, 'k', label='Nelder-Mead', markersize='7')   # predicted
plt.xlabel('wavelength (10nm)')
plt.ylabel('%Reflectance')
plt.title('Target Spectra vs Predicted spectra')
plt.legend(loc='best')
plt.grid(False )
plt.show()